# oobabooga/text-generation-webui

在运行所有单元格后，将在几分钟内在底部显示一个公共gradio URL。您还可以选择生成一个API链接。

* 项目地址：https://github.com/oobabooga/text-generation-webui
* Gradio服务器状态：https://status.gradio.app/

In [ ]:
#@title 1. 保持此标签页始终活动，防止Colab与你断开连接 { display-mode: "form" }

#@markdown 按下面的播放按钮：
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
#@title 2. 启动web UI

#@markdown 如果不确定branch，请填写“main”或留空。

import torch
from pathlib import Path

if Path.cwd().name != 'text-generation-webui':
  print("正在安装webui...")

  !git clone https://github.com/oobabooga/text-generation-webui
  %cd text-generation-webui

  torver = torch.__version__
  print(f"TORCH: {torver}")
  is_cuda118 = '+cu118' in torver  # 2.1.0+cu118

  if is_cuda118:
    !python -m pip install --upgrade torch==2.2.1 torchvision==0.17.1 torchaudio==2.2.1 --index-url https://download.pytorch.org/whl/cu118
  else:
    !python -m pip install --upgrade torch==2.2.1 torchvision==0.17.1 torchaudio==2.2.1 --index-url https://download.pytorch.org/whl/cu121

  textgen_requirements = open('requirements.txt').read().splitlines()
  if is_cuda118:
      textgen_requirements = [req.replace('+cu121', '+cu118').replace('+cu122', '+cu118') for req in textgen_requirements]
  with open('temp_requirements.txt', 'w') as file:
      file.write('\n'.join(textgen_requirements))

  !pip install -r temp_requirements.txt --upgrade

  print("\033[1;32;1m\n --> 如果你看到了关于 \"previously imported packages\" 的警告，请忽略它。\033[0;37;0m")
  print("\033[1;32;1m\n --> 无需重启运行时。\n\033[0;37;0m")

  try:
    import flash_attn
  except:
    !pip uninstall -y flash_attn

# Parameters
model_url = "https://huggingface.co/TheBloke/MythoMax-L2-13B-GPTQ" #@param {type:"string"}
branch = "gptq-4bit-32g-actorder_True" #@param {type:"string"}
command_line_flags = "--n-gpu-layers 128 --load-in-4bit --use_double_quant" #@param {type:"string"}
api = False #@param {type:"boolean"}

if api:
  for param in ['--api', '--public-api']:
    if param not in command_line_flags:
      command_line_flags += f" {param}"

model_url = model_url.strip()
if model_url != "":
    if not model_url.startswith('http'):
        model_url = 'https://huggingface.co/' + model_url

    # Download the model
    url_parts = model_url.strip('/').strip().split('/')
    output_folder = f"{url_parts[-2]}_{url_parts[-1]}"
    branch = branch.strip('"\' ')
    if branch.strip() not in ['', 'main']:
        output_folder += f"_{branch}"
        !python download-model.py {model_url} --branch {branch}
    else:
        !python download-model.py {model_url}
else:
    output_folder = ""

# Start the web UI
cmd = f"python server.py --share"
if output_folder != "":
    cmd += f" --model {output_folder}"
cmd += f" {command_line_flags}"
print(cmd)
!$cmd